## This notebook shows how to run a whole lot of propagations

In [1]:
import sys
from os.path import expanduser

# default adam location - alter directory structure if you decide to use a different path
# for example: adam_home_defined = expanduser("~") + "your_project_name/scenario1/adam_home"
adam_home_defined = expanduser("~") + "/adam_home" # default location

# Add adam path to system in order to import adam
sys.path.append(adam_home_defined)

# import adam and adam modules required
import adam
from adam import adam_config

In [2]:
from adam import Batch
from adam import PropagationParams
from adam import OpmParams
from adam import BatchRunManager
from adam import ConfigManager
from adam import Service

import datetime
import os
import pandas as pd

In [3]:
# Set up adam paths
ADAMpaths = adam_config.setPaths.initPaths(adam_home_defined)

Changing adam home path to =  /Users/macuser/adam_home


In [4]:
#Attempts to read config that is already set up from an adam_config.json file in adam_home/config directory.
#Feel free to point it somewhere else. If it fails to read that file, it will copy the template from adam.
config_file  = ADAMpaths[3]
template_file = ADAMpaths[2]

try:
    f = open(config_file)
    f.close()
except:
    copyfile(template_file, config_file)

This sets up a Service which uses the given token and URL to provide authorized access through the server through several wrapped modules. It also creates a project for you to work in that will be used for the rest of the notebook. Be sure to run service.teardown() when finished. 

If you don't have a config, see config_demo to get one.

In [5]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
# config = ConfigManager(os.getcwd() + '/config.json').get_config()
config_manager = ConfigManager(config_file)
config = config_manager.get_config()
service = Service(config)
service.setup()

[0.493794] Setup


True

In [6]:
working_project = service.new_working_project()

Set up project with uuid 6e67bb43-d54f-4a7c-82bb-f6a3ad49a8d9
[1.450103] Generate working project


This function creates a propagation object. For now, it just creates a dummy object with slight variations to avoid creating duplicate objects. For detailed information on Batch creation, see test_single_batch_run notebook.

In [7]:
def get_batch(i):
    state_vec = [130347560.13690618,
                 -74407287.6018632,
                 -35247598.541470632,
                 23.935241263310683,
                 27.146279819258538,
                 10.346605942591514]
    
    start_time = datetime.datetime(2018, 2, 21, 0, 0, 0, 123456 + i)
    end_time = datetime.datetime(2028, 2, 21, 0, 0, 0, 123456 + i)  # Ten years later.
    
    return Batch(PropagationParams({
        'start_time': start_time.isoformat() + 'Z',
        'end_time': end_time.isoformat() + 'Z',
        'project_uuid': working_project.get_uuid(),
    }), OpmParams({
        'epoch': start_time.isoformat() + 'Z',
        'state_vector': state_vec,
        'mass': 500 + i,  # Create a small variation between batches.
    }))

This creates a large number of batch runs and propagates them. This'll probably take ~10 minutes. It'll be faster to run the second time because the server is already all scaled up.

In [8]:
num_batches = 1000
batches = [get_batch(i) for i in range(num_batches)]

batch_run_manager = BatchRunManager(service.get_batches_module(), batches)
batch_run_manager.run()

[0.887784] Submitting 10 runs.
[31.218048] Running.
[2.446075] Retrieving propagation results.


TODO(laura): This is actually not true because python notebooks do not run cells simultaneously. There are ways to make this work. Do it.

To view the status of your runs while they're propagating, run this.

In [9]:
status = batch_run_manager.get_latest_statuses()
status_counts = [[s, len(status[s])] for s in status]
df = pd.DataFrame(status_counts, columns=['Task', 'Status'])
df.style.hide_index()

Task,Status
PENDING,0
RUNNING,0
COMPLETED,10
FAILED,0


Once the batches are all done propagating, you can view their results.

In [10]:
for batch in batches:
    if batch.get_calc_state() == 'FAILED':
        error = 'unknown error'
        if batch.get_results() is not None and batch.get_results().get_parts()[0] is not None:
            error = batch.get_results().get_parts()[0].get_error()
        #print('Batch %s failed: %s' % (batch.get_uuid(), error))
    else:
        final_state_vector = batch.get_results().get_end_state_vector()
        #print('Batch %s ended at %s' % (batch.get_uuid(), final_state_vector))#

In [11]:
batch_dict = { batch.get_uuid() : batch.get_results().get_end_state_vector() for batch in batches }
df = pd.DataFrame.from_dict(batch_dict,orient='index', columns=['Rx','Ry','Rz', 'Vx', 'Vy', 'Vz'])
df.style.set_caption('Batch UUID vs. Final State Vector')

,Rx,Ry,Rz,Vx,Vy,Vz
5684a8c5-8cf9-4cbe-acf1-b0f2f085b046,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
235ea4b8-0ef6-4346-adb2-5ed3bd96598d,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
6f7910a8-ac18-4274-8b67-b7e5c5495362,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
1e8ad900-e7f4-4eb1-bc24-c47a53f4b9ff,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
51912a35-70e8-4b62-8828-9197b25e9530,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
2ac0822f-9244-491a-ad8e-521456b1bf5e,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
0724f646-e47d-427b-9d23-d6df9386ae72,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
97b87360-4708-49c2-9a4b-07139db852b1,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
2b569512-5e7e-4c5b-80f5-da839131eb3b,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901
2ce77341-7ba0-451f-b4a6-001b7cd952de,1.73793e+08,3.00881e+06,-4.88971e+06,11.6743,30.583,12.1901


That's it! Please clean up by running the following:


In [12]:
batch_dict = { batch.get_uuid() : batch.get_results().get_parts()[0].get_error() for batch in batches }
df = pd.DataFrame.from_dict(batch_dict,orient='index', columns=['Error'])
df.style.set_caption('Batch UUID vs. Error')

,Error
5684a8c5-8cf9-4cbe-acf1-b0f2f085b046,None
235ea4b8-0ef6-4346-adb2-5ed3bd96598d,None
6f7910a8-ac18-4274-8b67-b7e5c5495362,None
1e8ad900-e7f4-4eb1-bc24-c47a53f4b9ff,None
51912a35-70e8-4b62-8828-9197b25e9530,None
2ac0822f-9244-491a-ad8e-521456b1bf5e,None
0724f646-e47d-427b-9d23-d6df9386ae72,None
97b87360-4708-49c2-9a4b-07139db852b1,None
2b569512-5e7e-4c5b-80f5-da839131eb3b,None
2ce77341-7ba0-451f-b4a6-001b7cd952de,None


In [13]:
service.teardown()

Cleaning up working project 6e67bb43-d54f-4a7c-82bb-f6a3ad49a8d9...
[0.973213] Teardown
